In [130]:

import copy
import gymnasium as gym
from typing import Generic, TypeVar, Tuple
import numpy as np
import torchrl
import torch as th

rl = torchrl.torchrl


In [131]:
# check render modes
env: rl.envs.EnvBase = rl.envs.GymEnv("CliffWalking-v0")
env


GymEnv(env=CliffWalking-v0, batch_size=torch.Size([]), device=cpu)

In [132]:
tensordict = env.reset()
tensordict


TensorDict(
    fields={
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        observation: Tensor(shape=torch.Size([48]), device=cpu, dtype=torch.int64, is_shared=False),
        terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        truncated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False)},
    batch_size=torch.Size([]),
    device=cpu,
    is_shared=False)

In [133]:
tensordict['observation']


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [134]:
def policy(tensordict, env=env):
    tensordict.set("action", env.action_spec.rand())
    return tensordict
policy(tensordict)
tensordict_out = env.step(tensordict)
tensordict_out


TensorDict(
    fields={
        action: Tensor(shape=torch.Size([4]), device=cpu, dtype=torch.int64, is_shared=False),
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([48]), device=cpu, dtype=torch.int64, is_shared=False),
                reward: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([]),
            device=cpu,
            is_shared=False),
        observation: Tensor(shape=torch.Size([48]), device=cpu, dtype=torch.int64, is_shared=False),
        terminated: T

In [135]:
from tensordict import TensorDict


In [151]:
def int_from_onehot(onehot: th.Tensor):
    return th.argmax(onehot)

def onehot_from_int(int, n=4, dtype=th.int64):
    onehot = th.zeros(n, dtype=dtype)
    onehot[int] = 1
    return onehot

def step_from_state(env, observation, action):
    return env.step(TensorDict(source={
        "action": action,
        "observation": observation,
    }, batch_size=[]))
reset = env.reset()

observation = onehot_from_int(0)
# take random action 5 times

for action in range(4):
    env.reset() # this makes a difference!! env is stateful :(
    print('action', action)
    tensordict = step_from_state(env, reset['observation'], onehot_from_int(action))
    print(int_from_onehot(tensordict['observation']))

    print(int_from_onehot(tensordict['next']['observation']))


action 0
tensor(36)
tensor(24)
action 1
tensor(36)
tensor(36)
action 2
tensor(36)
tensor(36)
action 3
tensor(36)
tensor(36)


In [152]:
# benchmark deepcopy of env
import time
env = rl.envs.GymEnv("CliffWalking-v0")
env.reset()
start = time.time()
for i in range(1000):
    env = copy.deepcopy(env)

print(time.time() - start)


2.809636116027832


In [157]:
# benchmakr with timeit
import timeit
timeit.timeit('copy.deepcopy(env)', globals=globals(), number=1000)


2.131824875017628